In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
filepath = '../input/home-data-for-ml-course/train.csv'
data = pd.read_csv(filepath)
data.info()

In [4]:
data.describe()


In [46]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
filepath = '../input/home-data-for-ml-course/train.csv'
data = pd.read_csv(filepath)
test_data = pd.read_csv('../input/home-data-for-ml-course/test.csv')
y = data.SalePrice

num_type = ['int64', 'float64']
impute_num = SimpleImputer(strategy = 'mean')
impute_cat = SimpleImputer(strategy = 'most_frequent')
train_cols_missing = [col for col in data.columns 
                                 if data[col].isnull().any()]
test_cols_missing = [col for col in test_data.columns if test_data[col].isnull().any()]
for col in data.columns:
    if data[col].dtype in num_type and col in train_cols_missing:
        data[col] = impute_num.fit_transform(data[[col]])
    elif data[col].dtype not in num_type and col in train_cols_missing:
        data[col] = impute_cat.fit_transform(data[[col]])
for col in test_data.columns:
    if test_data[col].dtype in num_type and col in test_cols_missing:
        test_data[col] = impute_num.fit_transform(test_data[[col]])
    elif test_data[col].dtype not in num_type and col in test_cols_missing:
        test_data[col] = impute_cat.fit_transform(test_data[[col]])

lbl=LabelEncoder()
for col in data.columns:
    if data[col].dtype not in num_type:
        data[col]=lbl.fit_transform(data[col])
for col in test_data.columns:
    if test_data[col].dtype not in num_type:
        test_data[col]=lbl.fit_transform(test_data[col])
        
        
model = RandomForestRegressor(random_state=2)

reduced_data = data.drop('SalePrice', axis=1)
#print(reduced_data)
#x_train,x_test,y_train,y_test=train_test_split(reduced_data,y,test_size=0.2,random_state=1)

model.fit(reduced_data,y)
y_pred = model.predict(test_data)
y_pred

In [47]:

from sklearn.metrics import accuracy_score
model.score(reduced_data,y)
model = round(model.score(reduced_data,y) * 100, 2)
model

In [53]:

final = pd.DataFrame({'Id':test_data['Id'].values,'SalePrice':y_pred})
print(final)
final.to_csv('/kaggle/working/submission.csv',index=False)